In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [129]:
class Clustering:
    def __init__(self, path='accounts_style.csv' ):
        self.path = path
        self.data = self.__pre_precessing(path)
        print(data)

    def __pre_precessing(self, path):
        #select special part of the dataframe
        data = pd.read_csv(path)
        data = data.iloc[ : ,2:]

        #spilit data and add new culomns
        data[['Age1', 'Age2', 'Age3']] = data["Age"].str.split(",", n=2, expand=True)
        data[['Color1', 'Color2', 'Color3']] = data["Color"].str.split(",", n=2, expand=True)
        data[['Pattern1', 'Pattern2', 'Pattern3']] = data["Pattern"].str.split(",", n=2, expand=True)
        data[['Size1', 'Size2', 'Size3']] = data["Size"].str.split(",", n=2, expand=True)
        data[['Formal1', 'Formal2', 'Formal3']] = data["Formal"].str.split(",", n=2, expand=True)

        #drop unusage columns
        data.drop(columns =['Age', 'Color', 'Pattern', 'Size', 'Formal'], inplace = True)

        #convert type from str to float64
        data = data.astype('float64')

        #calculate the average of each row specialy
        data['Age_avg'] = self.__special_mean(data[['Age1', 'Age2', 'Age3']].values,20)
        data['Color-avg'] = self.__special_mean(data[['Color1', 'Color2', 'Color3']].values,20)
        data['Pattern_avg'] = self.__special_mean(data[['Pattern1', 'Pattern2', 'Pattern3']].values,20)
        data['Size-avg'] = self.__special_mean(data[['Size1', 'Size2', 'Size3']].values,20)
        data['Formal_avg'] = self.__special_mean(data[['Formal1', 'Formal2', 'Formal3']].values,20)

        #drop unusage columns
        data.drop(columns =['Age1', 'Age2', 'Age3', 'Color1', 'Color2', 'Color3', 'Pattern1',
         'Pattern2', 'Pattern3', 'Size1', 'Size2', 'Size3', 'Formal1', 'Formal2', 'Formal3'], inplace = True)

        return data


    def __special_mean(self, selectedClothes, anomaly):
        # we have to ignore anomaly values in each component to have accessed the real mean value
        meanVector = []
        for i in range(np.shape(selectedClothes)[0]):
            componentSum = 0
            count = 0
            for j in range(np.shape(selectedClothes)[1]):
                if(selectedClothes[i][j] != anomaly):
                    componentSum += selectedClothes[i][j]
                    count += 1
            meanVector.append(componentSum/count)

        return meanVector




In [130]:
clustering = Clustering()


     Age_avg  Color-avg  Pattern_avg  Size-avg  Formal_avg
0        3.0        5.0          1.0       3.0         3.0
1        4.0        5.0          1.0       3.0         4.0
2        2.0        5.0          1.0       3.0         3.0
3        2.0        5.0          1.0       2.0         3.0
4        2.0        4.0          1.0       3.0         3.0
..       ...        ...          ...       ...         ...
395      2.5        2.5          1.0       2.0         3.5
396      2.5        5.0          3.0       2.0         2.5
397      2.5        4.5          3.0       2.5         1.5
398      2.5        2.5          3.0       2.0         2.0
399      2.5        5.0          3.0       2.5         2.5

[400 rows x 5 columns]
